So this is the lab...
Logistic regression and stuff
so yeah... we need paramters for regularization
none, L1, L2, this can jsut be a flag with hyperparamters and overloaded functions for the regulairzation...
well maybe not overloading but a method that we call that calls other methods called L1 and L2
we can compare different regularizations within the same method, that way its legit. 
for the methods... Were suggetsted to the Stochiastic graident descent. mini-batch gradient descent for our steepest descent and the hessian for the quasi newton thingy. Frick do we gotta write our own hessian? Thatll suck big time

Step1.
Read data and encode locations as one hot encoded
Step2.
Split into 80% train and 20 %test after merging.
Step3. Write regularization
Do these tonight


We did both of the regularzations but not together \for reg we do ridge lasso or elastic net 
gradient[1:] = regularization + get gradient

In [1]:
#this is the block that simply reads the data
import os
import re
import math
import pandas as pd
def translate_height(height):
    height = str(height)
    if height == "nan":
        return None
    l = re.findall(r'[0-9]+',height)
    if len(l) == 1:
        return int(l[0])
    else:
        return int(l[0]) + int(l[1])*12
directory = os.path.relpath("clothing-fit-dataset-for-size-recommendation/")
print(directory)
df = []
y = []
df = pd.read_json('clothing-fit-dataset-for-size-recommendation/modcloth_final_data.json', lines=True)
di = {"very short": 1, "slightly short": 2, "just right": 3, "slightly long": 4, "very long": 5}
df = df.replace({"length": di})
di = {"small": 1, "fit": 2, "large": 3}
df = df.replace({"fit": di})
df["height"] = df["height"].map(lambda x: translate_height(x))

di = {"aa": 0, "a": 1, "b": 2, "c": 3, "d": 4, "dd/e": 5, "ddd/f": 6, "dddd/g": 7, "h": 8, "i": 9, "j": 10, "k": 11}
df = df.replace({"cup size": di})
#maybe add shoe width back?
y = df['fit']
df = df.drop(columns=["item_id", "category", "user_name", "user_id", "shoe width", "review_summary", "review_text", "fit"])
print("NaNs per column, if more than 20% we will drop otherwise")
for column in df:
    print(df[column].isna().sum(), column)
    df[column] = pd.to_numeric(df[column], errors="coerce")
df = df.drop(columns=["waist", "bust"])
df = df.fillna(df.mean())
print(df.head())
#big issue here is predicting columns with shoes? wtf there are so many NaNs. Not sure what to do about shoe products?
x = df.values.astype(int)
x


clothing-fit-dataset-for-size-recommendation
NaNs per column, if more than 20% we will drop otherwise
6018 bra size
70936 bust
6255 cup size
1107 height
26726 hips
35 length
68 quality
54875 shoe size
0 size
79908 waist
    bra size  cup size     height       hips  length  quality  shoe size  size
0  34.000000       4.0  77.000000  38.000000     3.0      5.0   8.145818     7
1  36.000000       2.0  29.000000  30.000000     3.0      3.0   8.145818    13
2  32.000000       2.0  89.000000  40.358501     4.0      2.0   9.000000     7
3  35.972125       5.0  67.016136  40.358501     3.0      5.0   8.145818    21
4  36.000000       2.0  29.000000  40.358501     4.0      5.0   8.145818    18


array([[34,  4, 77, ...,  5,  8,  7],
       [36,  2, 29, ...,  3,  8, 13],
       [32,  2, 89, ...,  2,  9,  7],
       ...,
       [32,  7, 53, ...,  5,  8, 12],
       [35,  3, 41, ...,  4,  8, 12],
       [32,  4,  6, ...,  4,  8,  4]])

In [2]:
#this is for the renttherunway stuff
y

0        1
1        1
2        1
3        2
4        1
5        1
6        3
7        1
8        1
9        2
10       1
11       1
12       2
13       2
14       2
15       2
16       2
17       3
18       3
19       2
20       3
21       3
22       1
23       1
24       2
25       1
26       2
27       1
28       2
29       1
        ..
82760    2
82761    2
82762    1
82763    2
82764    2
82765    2
82766    2
82767    1
82768    2
82769    2
82770    1
82771    2
82772    2
82773    2
82774    2
82775    2
82776    2
82777    1
82778    1
82779    1
82780    2
82781    1
82782    2
82783    2
82784    3
82785    2
82786    1
82787    2
82788    2
82789    2
Name: fit, Length: 82790, dtype: int64

In [3]:
#now split into training and testing data
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .80)
y_train

55827    3
67216    2
52015    2
65463    2
45352    2
10419    2
69971    2
60396    2
43955    2
16500    2
14529    3
68738    2
18505    2
25409    2
71054    1
55078    3
78769    2
25284    1
67501    2
34063    1
47268    3
72856    1
52140    2
32485    2
34842    1
41066    2
75873    2
67354    2
16809    2
18140    2
        ..
35944    3
28869    2
18017    1
45625    1
7374     1
79298    2
77411    1
11408    2
32353    3
11382    2
25843    2
75196    2
34551    1
78765    2
53665    2
49774    2
45980    1
78770    2
48170    3
68959    2
67974    2
42516    3
48792    3
50344    2
2613     2
77143    1
50559    2
41960    2
55290    3
66477    2
Name: fit, Length: 16558, dtype: int64

In [4]:
#test logistic regresion to see if my splits are working
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression(fit_intercept=False)
clf.fit(X_train, y_train)

yhat = clf.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Accuracy of:  0.6921578693078875


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Regurlzarization


In [5]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # this typically gives us the probability of a specific class
        # we need a bias term 
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        # this will take the x with the bias term and multiplies it by w. This will give us a scalr quantity
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        # if we actually want to do a prediction we are seeing if it is greater than point 5. vector of ones were it is greater than .5 and 0 when it is below .5
        # When one is greater than the other it would be greater than .5
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [6]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        #for each vector x we want to out it throught he sigmoid suntract it from y and add it to the gradient function
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        #return gradient divided by the total number of points
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        #this is going to get features and labels. We are going to say x wtih a bias is ther (col of 1s)
        #the shape is the # of samples (rows ) by # of features (cols)
        #w always starts at 0 usually. 
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        # whats the gradient?? and internall we are updating self.w_ in this
        
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

           

In [7]:
from scipy.special import expit
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

In [8]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem whenever class is 0 (big class), y binary will be 1
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
#print(X)
#print(y)

In [9]:
#yhat = lr.predict(X_test)
#print('Accuracy of: ',accuracy_score(y_test,yhat))


In [10]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        # add in regularization (to all except bias term)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
        

In [11]:
# now redefine the Logistic Regression Function where needed
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [12]:
lr = RegularizedLogisticRegression(eta=0.1,
                                           iterations=2500,
                                           C=.4) # get object(0.1,500)
lr.fit(X_train,y_train)
print(lr)


Binary Logistic Regression Object with coefficients:
[[-6.77237347]
 [ 0.08450704]
 [ 0.02863585]
 [ 0.30670903]
 [ 0.08107187]
 [-0.042347  ]
 [-0.08944815]
 [ 0.00918663]
 [ 0.20468346]]
Binary Logistic Regression Object with coefficients:
[[ 2.43755601]
 [ 0.24052443]
 [-0.01880271]
 [ 0.2265909 ]
 [ 0.25971286]
 [ 0.0069501 ]
 [ 0.15843722]
 [ 0.07407142]
 [-0.4517289 ]]
Binary Logistic Regression Object with coefficients:
[[-6.87777301]
 [-2.78576053]
 [-0.2706531 ]
 [-5.16216076]
 [-3.1091138 ]
 [-0.17159901]
 [-0.33012429]
 [-0.62048979]
 [-0.82599739]]
MultiClass Logistic Regression Object with coefficients:
[[-6.77237347  0.08450704  0.02863585  0.30670903  0.08107187 -0.042347
  -0.08944815  0.00918663  0.20468346]
 [ 2.43755601  0.24052443 -0.01880271  0.2265909   0.25971286  0.0069501
   0.15843722  0.07407142 -0.4517289 ]
 [-6.87777301 -2.78576053 -0.2706531  -5.16216076 -3.1091138  -0.17159901
  -0.33012429 -0.62048979 -0.82599739]]


In [13]:
yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Accuracy of:  0.6300428795748279


In [14]:
#minibatch s essentially just running regression a smaller batch. This will be much faster with hopefully similar accuracy.
from numpy import random
class StochasticGradientDescent(RegularizedLogisticRegression):
    def __init__(self,size=1000, **kwds):
        self.size = size
            #make sure to pass c, eta and iterations
        super().__init__(**kwds) # call parent initializer
        
    def fit(self, X, y):
        num_samples, num_features = X.shape
        #get a random sample 
        x_slice = X[self.size:self.size*2,:]
        y_slice = y[self.size:self.size*2]
        print(y_slice)
        self.unique_ = np.unique(y_slice) # get each unique class value
        
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        #then we will
        print(np.unique(y_slice))
        #there are nans in here
        print(self.unique_)
        for i,yval in enumerate(self.unique_): # for each unique value
            print(i)
            print(yval)
            y_binary = y_slice==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                          C=self.C)
            blr.fit(x_slice,y_binary)
                # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        #stochastic takes a single sample 



In [15]:
#print(y_train)
#5194
ls = StochasticGradientDescent(eta=0.1,
                                           iterations=2500, size=1000,
                                           C=.4) # get object(0.1,500)

#print( len(y_train))

ls.fit(X_train,y_train)
#print(ls)


53244    2
66429    2
60796    2
31157    2
5550     3
81889    2
25567    2
43136    3
65731    2
73825    2
32222    1
13299    1
42954    3
32007    2
19901    2
73750    2
10349    2
57175    3
13176    2
45193    2
15677    2
50473    2
55898    3
63991    2
29715    3
76126    2
3495     3
58468    2
8291     1
37470    2
        ..
43835    3
14444    2
1587     2
48183    2
65031    2
37394    2
7917     2
9596     3
68266    2
63565    1
60707    1
70176    2
13204    2
45386    3
13246    2
13678    3
35626    2
35069    1
42412    3
50687    2
17136    2
79970    2
59363    2
16645    2
13499    1
49126    1
34313    2
32997    2
63086    3
61711    1
Name: fit, Length: 1000, dtype: int64
[1 2 3]
[1 2 3]
0
1
Binary Logistic Regression Object with coefficients:
[[-7.29999876]
 [-1.95341597]
 [-0.25602463]
 [-3.55668653]
 [-2.12581163]
 [-0.21406349]
 [-0.32502208]
 [-0.42525326]
 [-0.56535594]]
1
2
Binary Logistic Regression Object with coefficients:
[[ 3.44332077]
 [-0.35464

In [16]:
yhat = ls.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))


Accuracy of:  0.6878548133832588


In [33]:
class Newton(RegularizedLogisticRegression):
    def sig(theta):
        return 1/(1+np.exp(-theta)) 
    
    def __init__(self,size=1000, **kwds):
        self.size = size
            #make sure to pass c, eta and iterations
        super().__init__(**kwds) # call parent initializer
        
    def fit(self, X, y):
        print("SELF:\n",self)
        print("X:\n",X)
        print("Y:\n",y)
        
            
        S = diag(self.sig(X @ self.w_)*(1-self.sig(X @ self.w_)))
        hessian = (X.T*S*X)
        H_inv = np.linalg.inv(hessian)
        g = (X.T((self.sig(X @ self.w_)) - y))
        newt_update = H_inv * g
        
        
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T


In [34]:

ls = Newton(eta=0.1,
                                           iterations=2500, size=1000,
                                           C=.4) # get object(0.1,500)

#print( len(y_train))

ls.fit(X_train,y_train)
#print(ls)


SELF:
 Untrained MultiClass Logistic Regression Object
X:
 [[40  3 89 ...  5  9 12]
 [34  3 77 ...  3  8  8]
 [34  3 53 ...  5  8  8]
 ...
 [36  3 65 ...  5  8 20]
 [36  4 53 ...  4  8 12]
 [32  2 65 ...  4  8  6]]
Y:
 55827    3
67216    2
52015    2
65463    2
45352    2
10419    2
69971    2
60396    2
43955    2
16500    2
14529    3
68738    2
18505    2
25409    2
71054    1
55078    3
78769    2
25284    1
67501    2
34063    1
47268    3
72856    1
52140    2
32485    2
34842    1
41066    2
75873    2
67354    2
16809    2
18140    2
        ..
35944    3
28869    2
18017    1
45625    1
7374     1
79298    2
77411    1
11408    2
32353    3
11382    2
25843    2
75196    2
34551    1
78765    2
53665    2
49774    2
45980    1
78770    2
48170    3
68959    2
67974    2
42516    3
48792    3
50344    2
2613     2
77143    1
50559    2
41960    2
55290    3
66477    2
Name: fit, Length: 16558, dtype: int64


TypeError: sig() takes 1 positional argument but 2 were given